In [1]:

from tatd.utils import *
from tatd.read import *
from tatd.tatd import *

import argparse
torch.backends.cudnn.deterministic = True
torch.manual_seed(1234)

In [23]:
import time

import torch.optim as optim

import torch.distributed as dist

In [3]:
import multiprocessing
from joblib import Parallel, delayed
from tqdm import tqdm

In [5]:
name, sparse, rank, window, penalty, opt_scheme, lr, gpu, count = 'radar', 1, 10, 7, 0.1, 'als_adam', 0.01, 2, 10

In [6]:
loss_path, model_path, total_path  = get_path(name, sparse, rank, window,  penalty,
                                              opt_scheme, lr, count)
device = f'cuda:{gpu}'

dataset = read_dataset(name, device)
dataset['count'], dataset['window'] = count, window
nmode, ndim, tmode =  dataset['nmode'], dataset['ndim'], dataset['tmode']

density = dataset['ts_beta']

In [7]:
model = Tatd(nmode, ndim, tmode, density, rank, window, sparse, device).to(device)

print(f"START {name} sparse : {bool(sparse)} & {opt_scheme} : {lr} "
      f"Rank : {rank} Window : {window} Penalty : {penalty} \n")

START radar sparse : True & als_adam : 0.01 Rank : 10 Window : 7 Penalty : 0.1 



In [8]:
def metadata(train):
    indices = train.indices().cpu().numpy()
    vals = train.values().cpu().numpy()
    temp = np.vstack([indices, vals]).T

    dct = {}
    dct1 = {}

    for mode, index in enumerate(indices):
        old_c = 0
        dct[mode] = {}
        dct1[mode] = {}
        unique, indices, counts = np.unique(index, return_index = True, return_counts=True)
        for a, c, b in zip(* [unique, indices[1:], counts]):
            dct[mode][a] = b
            dct1[mode][a] = {}
            dct1[mode][a]['idx'] = {}
            dct1[mode][a]['value'] = {}
            nnz_entries = temp[old_c:c]
            old_c = c
            nnz_idxs = nnz_entries[:, :-1]
            nnz_vals = nnz_entries[:, -1]
            dct1[mode][a]['idx'] = nnz_idxs
            dct1[mode][a]['val'] = nnz_vals
#          'indoor_1', 'indoor_3', 'indoor_5', 'indoor_9',
#          'beijing_1', 'beijing_3', 'beijing_5', 'beijing_7',
#          'radar_1', 'radar_3', 'radar_5', 'radar_7', 'radar_9',
#          'mair_1', 'mair_3', 'mair_5',  'mair_7','mair_9']
​
name = 'ncfd'
file = f'../{path}/als_adam/{name}/best.txt'
df = pd.read_csv(file, sep = '\t')
df['Name'] = name
total = pd.DataFrame()
for name in names:
    if name.startswith('mair'):
        file = f'../{path}/als_adam2/{name}/best.txt'
    else:
        file = f'../{path}/als_adam/{name}/best.txt'
    df = pd.read_csv(file, sep = '\t')
    df['Name'] = name
    total = total.append(df)
total = total.reset_index(drop = True)
total.columns = ['Count', 'Iters', 'Time', 'Sparse', 'Rank', 'Window', 'Penalty', 'Scheme',
       'Lr', 'RMSE', 'MAE', 'Name']
total = total.dropna()
total = total[['Count', 'Iters', 'Sparse', 'Rank', 'Window', 'Penalty',
       'Scheme', 'Lr', 'RMSE', 'MAE', 'Name']]
​
total['Model'] = 'kernel'
total['model_file'] = total.apply(lambda row: f'../{path}/{row.Scheme}/{row.Name}/model/'
                                    f'{row.Sparse}_r_{row.Rank}_w_{row.Window}_'
                                    f'p_{float(row.Penalty)}_'
                                    f'lr_{float(row.Lr)}_'
                                    f'{row.Count}-{row.Iters}.pth.tar', axis=1)
​
total.loc[total['Name'] == 'mair_std', 'model_file', ] = total.apply(lambda row: f'../{path}/als_adam2/{row.Name}/model/'
                                    f'{row.Sparse}_r_{row.Rank}_w_{row.Window}_'
                                    f'p_{float(row.Penalty)}_'
                                    f'lr_{float(row.Lr)}_'
                                    f'{row.Count}-{row.Iters}.pth.tar', axis=1)
​
for i in [1, 3, 5, 7, 9]:
    total.loc[total['Name'] == f'mair_{i}', 'model_file', ] = total.apply(lambda row: f'../{path}/als_adam2/{row.Name}/model/'
                                    f'{row.Sparse}_r_{row.Rank}_w_{row.Window}_'
                                    f'p_{float(row.Penalty)}_'
                                    f'lr_{float(row.Lr)}_'
                                    f'{row.Count}-{row.Iters}.pth.tar', axis=1)
​
### HPC
# df = pd.read_csv('hpc.txt', sep = '\t',)
# df1 = pd.read_csv('new_hpc2.txt', sep = '\t',)
# df = df.append(df1)
​
df = pd.read_csv('uber_hpc.txt', sep = '\t',)
df['Sparse'] = 0
df['Lr'] = 0
df['Window'] = 0
df.columns = ['Name', 'Rank', 'Penalty', 'Count', 'RMSE', 'MAE', 'model_file', 'Scheme',
       'Model', 'Sparse', 'Lr', 'Window']
​
df.loc[lambda df: df['Name'] == 'energy1', 'Name'] = 'indoor'
df.loc[lambda df: df['Name'] == 'energy_1', 'Name'] = 'indoor_1'
df.loc[lambda df: df['Name'] == 'energy_3', 'Name'] = 'indoor_3'
df.loc[lambda df: df['Name'] == 'energy_5', 'Name'] = 'indoor_5'
df.loc[lambda df: df['Name'] == 'energy_9', 'Name'] = 'indoor_9'
​
​
total = total.append(df)
​
total = total.reset_index(drop = True)
​
total.to_csv('./data_ver1.txt', sep = '\t', header = True, index = False)
section 1 : Table
​
bdf = total[[ 'Model', 'Name', 'RMSE','MAE', 'Rank', 'Sparse' ]]
​
header = pd.MultiIndex.from_product([['beijing',  'mad', 'energy1', 'radar'],
                                     ['RMSE', 'MAE']],
                                    names=['Data', 'Metric'])
index = pd.MultiIndex.from_product([['kernel']],
                                    names=['Model'])
# index = pd.MultiIndex.from_product([['kernel', 'attention_dot']],
#                                     names=['Model'])
​
# index = pd.MultiIndex.from_product([['kernel', 'attention_dot', 'standard']],
#                                     names=['Model'])
​
​
​
​









Code

Python 3
​
from tatd.utils import *
from tatd.read import *
from tatd.tatd import *
​
import argparse
torch.backends.cudnn.deterministic = True
torch.manual_seed(1234)
<torch._C.Generator at 0x7efc4381d1f0>
import time
​
import torch.optim as optim
​
import torch.distributed as dist
import multiprocessing
from joblib import Parallel, delayed
from tqdm import tqdm
radar
name, sparse, rank, window, penalty, opt_scheme, lr, gpu, count = 'radar', 1, 10, 7, 0.1, 'als_adam', 0.01, 2, 10
loss_path, model_path, total_path  = get_path(name, sparse, rank, window,  penalty,
                                              opt_scheme, lr, count)
device = f'cuda:{gpu}'
​
dataset = read_dataset(name, device)
dataset['count'], dataset['window'] = count, window
nmode, ndim, tmode =  dataset['nmode'], dataset['ndim'], dataset['tmode']
​
density = dataset['ts_beta']
model = Tatd(nmode, ndim, tmode, density, rank, window, sparse, device).to(device)
​
print(f"START {name} sparse : {bool(sparse)} & {opt_scheme} : {lr} "
      f"Rank : {rank} Window : {window} Penalty : {penalty} \n")
START radar sparse : True & als_adam : 0.01 Rank : 10 Window : 7 Penalty : 0.1 

def metadata(train):
    indices = train.indices().cpu().numpy()
    vals = train.values().cpu().numpy()
    temp = np.vstack([indices, vals]).T
​
    dct = {}
    dct1 = {}
​
    for mode, index in enumerate(indices):
        old_c = 0
        dct[mode] = {}
        dct1[mode] = {}
        unique, indices, counts = np.unique(index, return_index = True, return_counts=True)
        for a, c, b in zip(* [unique, indices[1:], counts]):
            dct[mode][a] = b
            dct1[mode][a] = {}
            dct1[mode][a]['idx'] = {}
            dct1[mode][a]['value'] = {}
            nnz_entries = temp[old_c:c]
            old_c = c
            nnz_idxs = nnz_entries[:, :-1]
            nnz_vals = nnz_entries[:, -1]
            dct1[mode][a]['idx'] = nnz_idxs
            dct1[mode][a]['val'] = nnz_vals
    return dct, dct1

In [11]:
train, valid, test = dataset['train'], dataset['valid'], dataset['test']
nmode, tmode = dataset['nmode'], dataset['tmode']
window, count = dataset['window'], dataset['count']

dct, temp = metadata(train)
opt = optim.Adam([list(model.parameters())[tmode]], lr = lr)

In [46]:
als_train_model(dataset, model, penalty, opt_scheme, lr, rank, device,
                    loss_path, model_path, total_path)






  0%|          | 0/999 [00:00<?, ?it/s]

TypeError: ones(): argument 'size' must be tuple of ints, but found element of type dict at pos 1

In [36]:
dct[0][0]

5

In [43]:
# processed_list = Parallel(n_jobs=num_cores)(delayed(myfunction)(i,parameters) for i in inputs)
def least_square(model, train, penalty, mode, dct, temp, rank, device):
    num_cores = multiprocessing.cpu_count()
    dct = dct[mode]
    dct1 = temp[mode]
    Parallel(n_jobs = num_cores)(delayed(row_wise)(idx, rows, dct1, model, mode) for idx, rows in dct1.items())

In [44]:
def row_wise(idx, rows, dct, model, mode):
    ttime = time.time()
    with torch.no_grad():
        nnz = dct[idx]
        nnz_idxs = rows['idx']
        nnz_vals = rows['val']

        update = sub_function(nnz_idxs, nnz_vals, nnz, model, mode)

        model.factors[mode][idx] = update
    print(time.time()-ttime);

In [45]:
def sub_function(nnz_idxs, nnz_vals, nnz, model, mode):
    ones = torch.ones((nnz, rank)).to(device)
    for i, item in enumerate(zip(nnz_idxs.T, model.factors)):
        if i != mode:
            index, factor = item
            ones = ones * factor[torch.LongTensor(index).to(device), :]

    mat_b = torch.bmm(ones.view(nnz, rank, 1), ones.view(nnz, 1, rank))
    mat_b2 = torch.sum(mat_b, dim=0) + torch.eye(rank).to(device) * penalty

    vect_c = ones * torch.FloatTensor(nnz_vals).view(-1, 1).to(device)
    vect_c = torch.sum(vect_c, dim=0)

    ttime4 = time.time()

    update = torch.matmul(torch.inverse(mat_b2), vect_c)
    update = torch.where(torch.abs(update) < 0.000001, torch.zeros_like(update), update)
    
    return update


In [38]:
def least_square(model, train, penalty, mode, dct, temp, rank, device):
    dct = dct[mode]
    dct1 = temp[mode]
    for idx, rows in dct1.items():
        ttime = time.time()
        with torch.no_grad():
            nnz = dct[idx]
            nnz_idxs = rows['idx']
            nnz_vals = rows['val']
            
            print(nnz, nnz_idxs, nnz_vals)
            
            update = sub_function(nnz_idxs, nnz_vals, nnz, model)
                
            model.factors[mode][idx] = update
        print(time.time()-ttime);

In [7]:
def least_square(model, train, penalty, mode, dct, temp, rank, device):

    mode = 2
    nnz_count = dct[mode]
    idxn = len(nnz_count)
    idx_list = []
    val_list = []
    indices = nnz_count.keys()
    for idx, nnz in nnz_count.items():
        with torch.no_grad():
            nnz_entries = temp[temp[:, mode] == idx]
            nnz_idxs = nnz_entries[:, :-1]
            nnz_vals = nnz_entries[:, -1]
            idx_list.append(nnz_idxs)
            val_list.append(nnz_vals)



    print(idx_list[0])
    idx_list = np.array(idx_list)

    idx_list = torch.Tensor(idx_list)
    print(idx_list.shape)
    ones = torch.ones((idxn, nnz, rank)).to(device)
    for i, item in enumerate(zip(idx_list, model.factors)):
        if i != mode:
            index, factor = item
            ones = ones * factor[torch.LongTensor(index).to(device), :]

    mat_b = torch.bmm(ones.view(idxn, nnz, rank, 1), ones.view(idxn, nnz, 1, rank))
    # mat_b = torch.sum(mat_b, dim=0)
    # lambda = torch.stack(torch.eye(rank) * idxn).to(device)
    # mat_b2 = mat_b + lambda * 0.01
    #
    # vect_c = ones * torch.FloatTensor(nnz_vals).view(-1, 1).to(device)
    # vect_c = torch.sum(vect_c, dim=0)
    #
    # update = torch.matmul(torch.inverse(mat_b2), vect_c)
    # update = torch.where(torch.abs(update) < 0.000001, torch.zeros_like(update), update)
    # model.factors[mode][idx] = update

In [40]:


def als_train_model(dataset, model, penalty, opt_scheme, lr, rank, device,
                    loss_path, model_path, total_path):

    train, valid, test = dataset['train'], dataset['valid'], dataset['test']
    nmode, tmode = dataset['nmode'], dataset['tmode']
    window, count = dataset['window'], dataset['count']

    head = 'Iters\tTime\tTrnRMSE\tTrMAE\tValRMSE\tValMAE\n'
    with open(loss_path, 'w') as f:
        f.write(head)

    dct, temp = metadata(train)
    if opt_scheme == 'als_adam':
        opt = optim.Adam([list(model.parameters())[tmode]], lr = lr)
    else:
        opt = optim.SGD([list(model.parameters())[tmode]], lr=lr)

    start_time = time.time()
    old_rmse, inner_rmse, stop_iter, = 1e+5, 1e+5, 0
    for n_iter in trange(1, 1000):
        inner_num = 0
        stop = True
        for mode in range(1, nmode):
            if mode != tmode:
                least_square(model, train, penalty, mode, dct, temp, rank, device)
                trn_rmse, trn_mae = evaluate(model, train)
                val_rmse, val_mae = evaluate(model, valid)
                print('Iter', n_iter, 'Mode', mode, 'Train', trn_rmse, 'Valid', val_rmse)

In [27]:
import time
import torch
import torch.functional as F
import torch.optim as optim
import torch.nn as nn

from tqdm import tqdm
from tqdm import trange
from tatd.tatd import *
from tatd.utils import *
from tatd.train import *


In [5]:

def gradient(model, opt, train, penalty, tmode):

    opt.zero_grad()
    rec = model(train.indices())
    loss = (rec - train.values()).pow(2).sum()
    loss = loss + penalty * model.smooth_reg(tmode)
    print(f"Loss :{model.smooth_reg(tmode).data}, Penalty :{penalty}")
    loss.backward()
    opt.step()

    return loss

def metadata(train):
    indices = train.indices().cpu().numpy()
    vals = train.values().cpu().numpy()
    temp = np.vstack([indices, vals]).T

    dct = {}
    for mode, index in enumerate(indices):
        dct[mode] = {}
        unique, counts = np.unique(index, return_counts=True)
        for a, b in zip(* [unique, counts]):
            dct[mode][a] = b

    return dct, temp

# def least_square(model, train, penalty, mode, dct, temp, rank, device):

#     mode = 2
#     nnz_count = dct[mode]
#     idxn = len(nnz_count)
#     idx_list = []
#     val_list = []
#     indices = nnz_count.keys()
#     for idx, nnz in nnz_count.items():
#         with torch.no_grad():
#             nnz_entries = temp[temp[:, mode] == idx]
#             nnz_idxs = nnz_entries[:, :-1]
#             nnz_vals = nnz_entries[:, -1]
#             idx_list.append(nnz_idxs)
#             val_list.append(nnz_vals)



#     print(idx_list[0])
#     idx_list = np.array(idx_list)

#     idx_list = torch.Tensor(idx_list)
#     print(idx_list.shape)
#     ones = torch.ones((idxn, nnz, rank)).to(device)
#     for i, item in enumerate(zip(idx_list, model.factors)):
#         if i != mode:
#             index, factor = item
#             ones = ones * factor[torch.LongTensor(index).to(device), :]

#     mat_b = torch.bmm(ones.view(idxn, nnz, rank, 1), ones.view(idxn, nnz, 1, rank))
#     # mat_b = torch.sum(mat_b, dim=0)
#     # lambda = torch.stack(torch.eye(rank) * idxn).to(device)
#     # mat_b2 = mat_b + lambda * 0.01
#     #
#     # vect_c = ones * torch.FloatTensor(nnz_vals).view(-1, 1).to(device)
#     # vect_c = torch.sum(vect_c, dim=0)
#     #
#     # update = torch.matmul(torch.inverse(mat_b2), vect_c)
#     # update = torch.where(torch.abs(update) < 0.000001, torch.zeros_like(update), update)
#     # model.factors[mode][idx] = update

# def als_train_model(dataset, model, penalty, opt_scheme, lr, rank, device,
#                     loss_path, model_path, total_path):

#     train, valid, test = dataset['train'], dataset['valid'], dataset['test']
#     nmode, tmode = dataset['nmode'], dataset['tmode']
#     window, count = dataset['window'], dataset['count']

#     head = 'Iters\tTime\tTrnRMSE\tTrMAE\tValRMSE\tValMAE\n'
#     with open(loss_path, 'w') as f:
#         f.write(head)

#     dct, temp = metadata(train)
#     if opt_scheme == 'als_adam':
#         opt = optim.Adam([list(model.parameters())[tmode]], lr = lr)
#     else:
#         opt = optim.SGD([list(model.parameters())[tmode]], lr=lr)

#     start_time = time.time()
#     old_rmse, inner_rmse, stop_iter, = 1e+5, 1e+5, 0
#     for n_iter in trange(1, 1000):
#         inner_num = 0
#         stop = True
#         for mode in range(nmode):
#             if mode != tmode:
#                 least_square(model, train, penalty, mode, dct, temp, rank, device)
#                 trn_rmse, trn_mae = evaluate(model, train)
#                 val_rmse, val_mae = evaluate(model, valid)
#                 print('Iter', n_iter, 'Mode', mode, 'Train', trn_rmse, 'Valid', val_rmse)
#             else:
#                 model.turn_on_grad(mode)
#                 while(stop):
#                     stop = gradient(model, opt, train, penalty, tmode)
#                     trn_rmse, trn_mae = evaluate(model, train)
#                     val_rmse, val_mae = evaluate(model, valid)
#                     print('Iter', n_iter, 'Mode', mode, 'Train', trn_rmse, 'Valid', val_rmse)
# #                    print(val_rmse, inner_rmse)
# #                    if np.abs(val_rmse - inner_rmse) < 0.00001:
# #                        break
#                     if inner_num > 1:
#                         stop = False
#                     if val_rmse > inner_rmse:
#                         # break
#                         inner_num += 1
#                     inner_rmse = val_rmse
#                     if isNaN(trn_rmse):
#                         print("Nan break")
#                         break
#             trn_rmse, trn_mae = evaluate(model, train)
#             val_rmse, val_mae = evaluate(model, valid)

#         if val_rmse > old_rmse and n_iter >1:
#             stop_iter += 1
#         old_rmse = val_rmse

#         with open(loss_path, 'a') as f:
#             elapsed = time.time() - start_time
#             f.write(f'{n_iter:5d}\t{elapsed:.5f}\t')
#             f.write(f'{trn_rmse:.5f}\t{trn_mae:.5f}\t')
#             f.write(f'{val_rmse:.5f}\t{val_mae:.5f}\n')
#         if stop_iter == 1 or n_iter == 999:
#             te_rmse, te_mae = evaluate(model, test)
#             with open(total_path, 'a') as f1:
#                 f1.write(f'{count:5d}\t{n_iter:5d}\t{elapsed:.3f}\t{model.sparse}\t')
#                 f1.write(f'{model.factors[0].shape[1]:2d}\t')
#                 f1.write(f'{window:2d}\t{penalty:.3f}\t')
#                 f1.write(f'{opt_scheme}\t{lr:5f}\t')
#                 f1.write(f'{te_rmse:.5f}\t{te_mae:.5f}\n')

#             p = f'{model_path}-{n_iter}.pth.tar'
#             save_checkpoints(model, p)
#             break
